# Data Collection

The first step is to get the data we will be working with from Kaggle.

Gonna use this one

https://www.kaggle.com/datasets/alanvourch/tmdb-movies-daily-updates

it has a ton of data in it, about 240mb. We can do a lot with this.

In [1]:
%pip -q install pandas openai python-dotenv chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
# Load the csv in ../raw/TMDB_all_movies.csv
import pandas as pd


In [ ]:

df = pd.read_csv('../raw/TMDB_all_movies.csv')

df.head()

In [4]:
# Print the shape of the dataframe
print(df.shape)

(992715, 27)


In [5]:
# Print the columns of the dataframe
print(df.columns)
# Print the dtypes of the dataframe
print(df.dtypes)
# Print the info of the dataframe
print(df.info())
# Print the description of the dataframe
print(df.describe())

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'cast', 'director', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'imdb_rating', 'imdb_votes'],
      dtype='object')
id                           int64
title                       object
vote_average               float64
vote_count                 float64
status                      object
release_date                object
revenue                    float64
runtime                    float64
budget                     float64
imdb_id                     object
original_language           object
original_title              object
overview                    object
popularity                 float64
tagline                     object
genres                  

In [7]:
# Print a row
print(df.iloc[0])


id                                                                         2
title                                                                  Ariel
vote_average                                                             7.1
vote_count                                                             327.0
status                                                              Released
release_date                                                      1988-10-21
revenue                                                                  0.0
runtime                                                                 73.0
budget                                                                   0.0
imdb_id                                                            tt0094675
original_language                                                         fi
original_title                                                         Ariel
overview                   After the coal mine he works at closes and his...

In [15]:
# Sort by date and find first and last date
df = df.sort_values(by='release_date', ascending=False)
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
268002,401563,100 Years,0.0,0.0,Post Production,2115-11-18,0.0,0.0,0.0,tt5174640,...,United States of America,English,"Shuya Chang, Marko Zaror, John Malkovich",Robert Rodriguez,Claudio Miranda,John Malkovich,"Robert Rodriguez, Gaspard Chevance",NaN,NaN,NaN
735514,1035190,The VelociPastor 2,0.0,0.0,Post Production,2099-12-31,0.0,0.0,275000.0,tt25404142,...,United States of America,English,"Alyssa Kempinski, Gregory James Cohan",Jesse Gouldsbury,NaN,"Jesse Gouldsbury, Brendan Steere","Alyssa Kempinski, Gregory James Cohan, Jesse G...",NaN,NaN,NaN
978438,1340871,O dia mundo Jurassic: O Filme,0.0,0.0,In Production,2034-12-15,0.0,0.0,0.0,NaN,...,NaN,Português,NaN,Audry Miguel,NaN,NaN,NaN,NaN,NaN,NaN
261170,393209,Avatar 5,0.0,0.0,In Production,2031-12-17,0.0,0.0,600000000.0,tt5637536,...,United States of America,English,"Matt Gerald, Oona Chaplin, Jemaine Clement, Mi...",James Cameron,NaN,James Cameron,"James Cameron, Jon Landau",Simon Franglen,NaN,NaN
923905,1279468,War of the Winds III,0.0,0.0,Rumored,2031-06-06,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Get all possible options in status column
df['status'].unique()

array(['Post Production', 'In Production', 'Rumored', 'Planned',
       'Canceled', 'Released', nan], dtype=object)

In [20]:
released = df[df['status'] == 'Released']
print(released.shape)
released.head(100)


(976249, 27)


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
926675,1282492,The Uncanny X-Men,0.0,0.0,Released,2026-11-06,0.0,0.0,0.0,NaN,...,NaN,NaN,"Taron Egerton, Alexandra Daddario, Josh Hollow...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
218593,340374,The Gettysburg Address,0.0,0.0,Released,2025-11-19,0.0,165.0,0.0,tt3066808,...,United States of America,English,"Jason Alexander, Lois Smith, Cary Elwes, Victo...",Sean Conant,NaN,Sean Conant,"Sean Conant, Jim Conant",NaN,NaN,NaN
774919,1092506,On Swift Horses,0.0,0.0,Released,2025-07-25,0.0,117.0,0.0,tt15042300,...,United States of America,English,"Jacob Elordi, Sasha Calle, Daisy Edgar-Jones, ...",Daniel Minahan,Luc Montpellier,"Shannon Pufahl, Bryce Kass","Claude Amadeo, David Darby, Jacob Elordi, Mich...",Mark Orton,6.6,63.0
894116,1245728,Peacock,0.0,0.0,Released,2025-07-02,0.0,90.0,0.0,tt29730305,...,"Austria, Germany",Deutsch,"Maria Fliri, Sabine Herget, Herbert Forthuber,...",Bernhard Wenger,Albin Wildner,Bernhard Wenger,"Michael Kitzberger, Markus Glaser, Nikolaus Ge...",NaN,6.6,18.0
831695,1169789,Bob Trevino Likes It,0.0,0.0,Released,2025-06-13,0.0,102.0,0.0,tt28613536,...,United States of America,English,"Barbie Ferreira, Rachel Bay Jones, John Leguiz...",Tracie Laymon,John Rosario,Tracie Laymon,"Kevin Forrester, Matthew Helderman, Felipe Die...",Jacques Brautbar,8.3,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977125,1339410,Filhos... para que tê-los?,0.0,0.0,Released,2024-12-01,0.0,0.0,0.0,NaN,...,NaN,NaN,"Sandra Lúcia, Daniel Ricardo",Daniel Ricardo,NaN,Daniel Ricardo,Daniel Ricardo,NaN,NaN,NaN
834753,1173559,Insidious Inferno,0.0,0.0,Released,2024-11-29,0.0,97.0,0.0,tt26349732,...,United States of America,English,NaN,Calvin Morie McCarthy,NaN,Calvin Morie McCarthy,NaN,NaN,2.8,185.0
835533,1174487,Escanyapobres,0.0,0.0,Released,2024-11-29,0.0,102.0,0.0,tt19177354,...,Spain,Català,"Quim Àvila Conde, Alfred Picó, Pep Molina, Jul...",Ibai Abad,NaN,"Elisenda Gorgues, Ibai Abad","Marc Roma, Jordi Llorca Llinares, Araceli Isaa...",NaN,7.9,18.0
785386,1106814,Minghun,0.0,0.0,Released,2024-11-29,0.0,94.0,0.0,tt15538962,...,Poland,"English, 普通话, Polski","Ewelina Żak, Wiktoria Gorodeckaja, Mateusz Web...",Jan P. Matuszyński,Kacper Fertacz,"Grzegorz Łoszewski, Jan Topczewski, Krzysztof Rak","Inga Kruk, Klaudiusz Frydrych, Krzysztof Rak",Stefan Wesołowski,NaN,NaN


In [21]:
# Get rows where the release_date is before 2025
before_2025 = released[released['release_date'] < '2025-01-01']


In [29]:
after_1950 = before_2025[before_2025['release_date'] > '1950-01-01']

print(after_1950.shape)
after_1950.head()

(812402, 27)


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
896229,1248196,Saba,0.0,0.0,Released,2024-12-31,0.0,95.0,0.0,tt31464877,...,Bangladesh,বাংলা,"Golam Shahriar Shikto, Shirin Alam, Mostafa Mo...",Maksud Hossain,Barkat Hossain Polash,"Maksud Hossain, Trilora Khan","Trilora Khan, Tamim Majid, Maksud Hossain, Meh...",Amman Abbasi,NaN,NaN
873645,1221051,The Way We Dream,0.0,0.0,Released,2024-12-31,0.0,63.0,0.0,tt31722973,...,NaN,NaN,NaN,"Brighton Grace, Daniel Zander, Guillermo Mijar...",NaN,NaN,NaN,NaN,NaN,NaN
868113,1214457,Gaucho Gaucho,0.0,0.0,Released,2024-12-27,0.0,85.0,0.0,tt30319823,...,"Argentina, Greece, United States of America",Español,"Guada Gonza, Tati Gonza, Solano Avalos, Jony A...","Gregory Kershaw, Michael Dweck","Gregory Kershaw, Michael Dweck",NaN,"Jamie Wolf, Susan Swartz, Cameron O’Reilly, Re...",NaN,7.6,38.0
756333,1064758,Planet B,0.0,0.0,Released,2024-12-25,0.0,118.0,0.0,tt28636876,...,"Belgium, France",Français,"Marc Barbé, Amine Hamidou, Luana Duchemin, Eli...",Aude Léa Rapin,Jeanne Lapoirie,"Laurent Lunetta, Aude Léa Rapin",Eve Robin,NaN,6.4,9.0
974567,1336546,Team Havnaa,0.0,0.0,Released,2024-12-25,0.0,0.0,0.0,NaN,...,NaN,NaN,"Tobias Santelmann, Odin Waage",Bård Breien,NaN,"Bård Breien, Eske Troelstrup, Ravn Lanesskog",NaN,NaN,NaN,NaN


In [30]:
# movies with a vote count of more than 0
voted_on = after_1950[after_1950['vote_count'] > 0]
print(voted_on.shape)
voted_on.head()

(322683, 27)


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
848582,1190698,"To Live, To Die, To Live Again",6.0,1.0,Released,2024-09-25,0.0,109.0,0.0,tt30426575,...,France,Français,"Amanda Lear, Elli Medeiros, Victor Belmondo, T...",Gaël Morel,NaN,"Gilles Taurand, Laurette Polmanss, Gaël Morel",NaN,NaN,6.8,25.0
416665,592831,Megalopolis,4.0,1.0,Released,2024-09-25,0.0,138.0,120000000.0,tt10128846,...,United States of America,English,"Laurence Fishburne, Isabelle Kusman, Dustin Ho...",Francis Ford Coppola,Mihai Malaimare Jr.,Francis Ford Coppola,"Barrie M. Osborne, Darren M. Demetre, Barry J....","Osvaldo Golijov, Grace VanderWaal",6.4,1508.0
868710,1215162,Kill 'em All 2,4.0,1.0,Released,2024-09-23,0.0,85.0,0.0,tt6549510,...,"Italy, United States of America","English,","María Conchita Alonso, Talia Asseraf, Andrei L...",Valeri Milev,NaN,"Jim Agnew, Ferdinando Dell'Omo, Andrea Iervolino","Richard Salvatore, Rafael Primorac, Monika Bac...",NaN,NaN,NaN
762460,1074506,A Sirens Call,8.0,1.0,Released,2024-09-21,0.0,21.0,10000.0,tt14220916,...,United States of America,English,"Tamar Shani, Danilo Lazo, Stanislav Tabirta, J...",Adam Moses,NaN,"Sof Puchley, Caroline Rabin","Adam Moses, Andrew Babicki, Michael Dutzer, Al...",NaN,8.9,8.0
921479,1276825,Long Gone Heroes,6.0,2.0,Released,2024-09-20,0.0,92.0,18000000.0,tt6001846,...,"Colombia, United States of America",English,"Beau Knapp, Mekhi Phifer, Wendy Moniz, Juan Pa...",John Swab,Brandon Cox,"Santiago Manes Moreno, John Swab","Noli Molla, Cristina Villar, Santiago Manes Mo...",David Sardy,NaN,NaN


In [36]:
# Sort by vote count
voted_on = voted_on.sort_values(by='vote_count', ascending=False)
voted_on.head()


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes
16347,27205,Inception,8.369,36269.0,Released,2010-07-15,8.255328e+08,148.0,160000000.0,tt1375666,...,"United Kingdom, United States of America","English, Français, 日本語, Kiswahili","Leonardo DiCaprio, Tim Kelleher, Jack Gilroy, ...",Christopher Nolan,Wally Pfister,Christopher Nolan,"Thomas Tull, Yoshikuni Taki, Christopher Nolan...",Hans Zimmer,8.8,2593927.0
96741,157336,Interstellar,8.440,35193.0,Released,2014-11-05,7.017292e+08,169.0,165000000.0,tt0816692,...,"United Kingdom, United States of America",English,"Russ Fega, Liam Dickinson, Jeff Hephner, Brook...",Christopher Nolan,Hoyte van Hoytema,"Jonathan Nolan, Christopher Nolan","Jake Myers, Jordan Goldberg, Lynda Obst, Chris...",Hans Zimmer,8.7,2159078.0
116,155,The Dark Knight,8.516,32550.0,Released,2008-07-16,1.004558e+09,152.0,185000000.0,tt0468569,...,"United Kingdom, United States of America","English, 普通话","Matthew O'Neill, Nestor Carbonell, Morgan Free...",Christopher Nolan,Wally Pfister,"David S. Goyer, Jonathan Nolan, Bob Kane, Chri...","Kevin De La Noy, Michael Uslan, Thomas Tull, C...","Hans Zimmer, James Newton Howard",9.0,2921100.0
12058,19995,Avatar,7.582,31265.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,...,"United States of America, United Kingdom","English, Español","Jason Whyte, Scott Lawrence, Brandon Jelkes, J...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,7.9,1402405.0
178666,293660,Deadpool,7.621,30696.0,Released,2016-02-09,7.828373e+08,108.0,58000000.0,tt1431045,...,United States of America,English,"Ryan Reynolds, Gina Carano, Kayla Adams, Randa...",Tim Miller,Ken Seng,"Paul Wernick, Fabian Nicieza, Rob Liefeld, Rhe...","Ryan Reynolds, Jonathon Komack Martin, Paul We...",Tom Holkenborg,8.0,1167674.0


In [37]:
top_by_vote_count = voted_on.head(150000)

### Embeddings

Okay now we need to get the embeddings

We are going to embed just the title and overview for now using openai embeddings

In [38]:
df = top_by_vote_count

In [40]:
# Get a list of all of the titles their overviews
titles = df['title'].tolist()
overviews = df['overview'].tolist()

titles_and_overviews = [f"{title} {overview}" for title, overview in zip(titles, overviews)]

print(titles_and_overviews[0])


Inception Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.


In [41]:
from openai import OpenAI
import openai
from tqdm.notebook import tqdm
from math import ceil

client = OpenAI()

def chunkify(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


In [42]:
# Parameters
chunk_size = 2048
model_name = "text-embedding-3-large"  # Ensure this is the correct model name

# Initialize list to store embeddings
all_embeddings = []

# Calculate total number of chunks
total_chunks = ceil(len(titles_and_overviews) / chunk_size)

# Process each chunk
for chunk in chunkify(titles_and_overviews, chunk_size):
    try:
        response = client.embeddings.create(
            input=chunk,
            model=model_name
        )
        # Extract embeddings from the response and append to the list
        embeddings = [embedding.embedding for embedding in response.data]
        all_embeddings.extend(embeddings)
        print(f"Processed {len(all_embeddings)} embeddings out of {len(titles_and_overviews)}")
    except openai.OpenAIError as e:
        print(f"An error occurred: {e}")
        # Optionally, implement retry logic or handle the error as needed
        raise e

# Verify the number of embeddings matches the number of documents
assert len(all_embeddings) == len(
    titles_and_overviews), "Mismatch between embeddings and documents."

# Now, all_embeddings[i] corresponds to titles_and_overviews[i]
print("All embeddings generated successfully.")

Processed 2048 embeddings out of 150000
Processed 4096 embeddings out of 150000
Processed 6144 embeddings out of 150000
Processed 8192 embeddings out of 150000
Processed 10240 embeddings out of 150000
Processed 12288 embeddings out of 150000
Processed 14336 embeddings out of 150000
Processed 16384 embeddings out of 150000
Processed 18432 embeddings out of 150000
Processed 20480 embeddings out of 150000
Processed 22528 embeddings out of 150000
Processed 24576 embeddings out of 150000
Processed 26624 embeddings out of 150000
Processed 28672 embeddings out of 150000
Processed 30720 embeddings out of 150000
Processed 32768 embeddings out of 150000
Processed 34816 embeddings out of 150000
Processed 36864 embeddings out of 150000
Processed 38912 embeddings out of 150000
Processed 40960 embeddings out of 150000
Processed 43008 embeddings out of 150000
Processed 45056 embeddings out of 150000
Processed 47104 embeddings out of 150000
Processed 49152 embeddings out of 150000
Processed 51200 embe

In [44]:
# Copy df and add the embeddings
df_with_embeddings = df.copy()
df_with_embeddings['embedding'] = all_embeddings




In [53]:
# Remove the id column and the index column and replace them with i in order of vote count
ided_df = df_with_embeddings.reset_index().drop(columns=['id'])
ided_df = ided_df.drop(columns=['index'])
ided_df.head()

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding
0,Inception,8.369,36269.0,Released,2010-07-15,8.255328e+08,148.0,160000000.0,tt1375666,en,...,"English, Français, 日本語, Kiswahili","Leonardo DiCaprio, Tim Kelleher, Jack Gilroy, ...",Christopher Nolan,Wally Pfister,Christopher Nolan,"Thomas Tull, Yoshikuni Taki, Christopher Nolan...",Hans Zimmer,8.8,2593927.0,"[-0.007635640446096659, -0.004168331157416105,..."
1,Interstellar,8.440,35193.0,Released,2014-11-05,7.017292e+08,169.0,165000000.0,tt0816692,en,...,English,"Russ Fega, Liam Dickinson, Jeff Hephner, Brook...",Christopher Nolan,Hoyte van Hoytema,"Jonathan Nolan, Christopher Nolan","Jake Myers, Jordan Goldberg, Lynda Obst, Chris...",Hans Zimmer,8.7,2159078.0,"[-0.0350022129714489, 0.023837948217988014, -0..."
2,The Dark Knight,8.516,32550.0,Released,2008-07-16,1.004558e+09,152.0,185000000.0,tt0468569,en,...,"English, 普通话","Matthew O'Neill, Nestor Carbonell, Morgan Free...",Christopher Nolan,Wally Pfister,"David S. Goyer, Jonathan Nolan, Bob Kane, Chri...","Kevin De La Noy, Michael Uslan, Thomas Tull, C...","Hans Zimmer, James Newton Howard",9.0,2921100.0,"[0.002534319879487157, -0.03606430068612099, -..."
3,Avatar,7.582,31265.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,en,...,"English, Español","Jason Whyte, Scott Lawrence, Brandon Jelkes, J...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,7.9,1402405.0,"[-0.028900334611535072, -0.0029624924063682556..."
4,Deadpool,7.621,30696.0,Released,2016-02-09,7.828373e+08,108.0,58000000.0,tt1431045,en,...,English,"Ryan Reynolds, Gina Carano, Kayla Adams, Randa...",Tim Miller,Ken Seng,"Paul Wernick, Fabian Nicieza, Rob Liefeld, Rhe...","Ryan Reynolds, Jonathon Komack Martin, Paul We...",Tom Holkenborg,8.0,1167674.0,"[-0.05629737302660942, 0.005468115210533142, -..."


In [54]:
# Pickle the dataframe
ided_df.to_pickle('../raw/df_with_embeddings.pkl')

In [4]:
# Assuming you're using a chroma db client for semantic search
from chromadb import Client


def get_embeddings(inputs: list[str], model_name: str = "text-embedding-3-large"):
    response = client.embeddings.create(
        input=inputs,
        model=model_name
    )
    return [embedding.embedding for embedding in response.data]


def load_df(file_path: str = '../raw/df_with_embeddings.pkl'):
    return pd.read_pickle(file_path)


class MovieData:
    def __init__(self, movies_df: pd.DataFrame):
        self.df = movies_df
        batch_size = 41666

        print("Loading vector db")
        self.db_client = Client()  # Assuming chromadb client initialization

        # If collection exists, use it
        try:
            self.collection = self.db_client.get_collection(name="movies")
            entries = self.collection.count()
            if entries != len(self.df):
                print("Collection does not match the number of movies")
                raise ValueError(
                    "Collection does not match the number of movies")
        except:
            self.collection = self.db_client.get_or_create_collection(
                name="movies")

            ids = self.df.index.tolist()
            id_strings = [str(id) for id in ids]
            embeddings = self.df['embedding'].tolist()

            for i in range(0, len(ids), batch_size):
                print(f"Processing batch {i} of {len(ids)}")
                batch_ids = id_strings[i:i + batch_size]
                batch_embeddings = embeddings[i:i + batch_size]
                self.collection.upsert(
                    ids=batch_ids, embeddings=batch_embeddings)

    def semantic_search(self, query: str, k: int = 10):
        response = client.embeddings.create(
            input=query,
            model=model_name
        )
        query_embedding = response.data[0].embedding
        results = self.collection.query(
            query_embeddings=[query_embedding], n_results=k)
        ids: list[str] = results['ids'][0]
        # df from the ids
        ids = [int(i) for i in ids]

        results_df = self.df.loc[ids]
        results_df["distance"] = results['distances'][0]
        return results_df


# movie_data = MovieData(ided_df)

In [90]:

movie_data.semantic_search("Ghosts of a relative")

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding,distance
147820,Ashes,7.000,3.0,Released,2018-10-06,0.0,93.0,200000.0,tt8928500,en,...,"Maria Olsen, Yumarie Morales, Emma Mumford, Me...",Barry Jay,NaN,Barry Jay,NaN,NaN,3.8,364.0,"[-0.030244959518313408, -0.03005353733897209, ...",0.945694
100393,The Ghost,7.700,7.0,Released,2023-09-29,0.0,20.0,0.0,tt28834550,en,...,"Andie Ju, Eddie Shin, Tania Verafield, Hana Ma...",Erica Eng,NaN,Kevin Jihyek Park,"Phillip Domfeh, Kelly Rutta",Jessica Rae Huber,6.1,113.0,"[-0.03387627378106117, -0.03882415220141411, -...",0.956890
129163,A Few Pages From the Life of a Ghost,5.800,4.0,Released,1992-12-31,0.0,15.0,0.0,tt0425290,ru,...,"Rogvold Sukhoverko, Vsevolod Abdulov",Marianna Novogrudskaya,NaN,NaN,NaN,NaN,6.2,20.0,"[-0.0152492206543684, -0.029503604397177696, -...",0.991295
39831,"Ghosts, Italian Style",6.283,30.0,Released,1967-12-23,0.0,104.0,0.0,tt0063480,it,...,"Valentino Macchi, Sophia Loren, Margaret Lee, ...",Renato Castellani,Tonino Delli Colli,"Leonardo Benvenuti, Adriano Baracco, Renato Ca...",Carlo Ponti,Luis Bacalov,5.5,598.0,"[-0.020237041637301445, -0.012776413932442665,...",0.999193
25684,Ghosts of Rome,7.202,62.0,Released,1961-04-01,0.0,96.0,0.0,tt0054867,it,...,"Bruno Scipioni, Marcello Mastroianni, Antonell...",Antonio Pietrangeli,Giuseppe Rotunno,"Sergio Amidei, Ettore Scola, Ruggero Maccari, ...",Franco Cristaldi,Nino Rota,6.8,683.0,"[-0.0028774267993867397, -0.02413448877632618,...",1.005611
147331,1995,6.700,3.0,Released,2020-03-30,0.0,6.0,0.0,NaN,en,...,Krishan Tait,Clémence Agenet,NaN,Clémence Agenet,NaN,NaN,NaN,NaN,"[-0.010433888994157314, -0.01415774505585432, ...",1.014267
144773,Ghosting,8.800,4.0,Released,2015-01-10,0.0,90.0,0.0,tt3349728,en,...,"Paul Fahrenkopf, Ralph Mauriello, Khawaja Aziz...",Kevin Alexander Boon,NaN,"Kevin Alexander Boon, Dalena Ly, Christopher W...",Kevin Alexander Boon,NaN,5.1,140.0,"[0.00841318629682064, -0.03602926805615425, -0...",1.015846
149964,Shadow of a Ghost,5.000,3.0,Released,1981-01-01,0.0,98.0,0.0,tt5820782,zh,...,"Jiang Yishou, Huang Ling, Chi Zhiqiang, Yuan Y...","Guo Baochang, Huang Ling",NaN,计红绪,NaN,NaN,NaN,NaN,"[-0.005402904003858566, -0.029550252482295036,...",1.016760
130959,Mi fantasma y yo,4.750,4.0,Released,1988-10-27,0.0,90.0,0.0,tt0244657,es,...,"Miguel Ángel Rodríguez, Xavier López, Valentín...",Gilberto de Anda,Antonio de Anda,Gilberto de Anda,NaN,NaN,6.8,26.0,"[-0.00021046862821094692, -0.02342719584703445...",1.021723
43736,Ghost,5.900,26.0,Released,1984-01-01,0.0,6.0,0.0,tt0087329,ja,...,NaN,Takashi Ito,NaN,NaN,NaN,NaN,7.2,620.0,"[0.0024297810159623623, -0.032166045159101486,...",1.023538


In [5]:
full_df = load_df()
full_df.head()

,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding
0,Inception,8.369,36269.0,Released,2010-07-15,8.255328e+08,148.0,160000000.0,tt1375666,en,...,"English, Français, 日本語, Kiswahili","Leonardo DiCaprio, Tim Kelleher, Jack Gilroy, ...",Christopher Nolan,Wally Pfister,Christopher Nolan,"Thomas Tull, Yoshikuni Taki, Christopher Nolan...",Hans Zimmer,8.8,2593927.0,"[-0.007635640446096659, -0.004168331157416105,..."
1,Interstellar,8.440,35193.0,Released,2014-11-05,7.017292e+08,169.0,165000000.0,tt0816692,en,...,English,"Russ Fega, Liam Dickinson, Jeff Hephner, Brook...",Christopher Nolan,Hoyte van Hoytema,"Jonathan Nolan, Christopher Nolan","Jake Myers, Jordan Goldberg, Lynda Obst, Chris...",Hans Zimmer,8.7,2159078.0,"[-0.0350022129714489, 0.023837948217988014, -0..."
2,The Dark Knight,8.516,32550.0,Released,2008-07-16,1.004558e+09,152.0,185000000.0,tt0468569,en,...,"English, 普通话","Matthew O'Neill, Nestor Carbonell, Morgan Free...",Christopher Nolan,Wally Pfister,"David S. Goyer, Jonathan Nolan, Bob Kane, Chri...","Kevin De La Noy, Michael Uslan, Thomas Tull, C...","Hans Zimmer, James Newton Howard",9.0,2921100.0,"[0.002534319879487157, -0.03606430068612099, -..."
3,Avatar,7.582,31265.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,en,...,"English, Español","Jason Whyte, Scott Lawrence, Brandon Jelkes, J...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,7.9,1402405.0,"[-0.028900334611535072, -0.0029624924063682556..."
4,Deadpool,7.621,30696.0,Released,2016-02-09,7.828373e+08,108.0,58000000.0,tt1431045,en,...,English,"Ryan Reynolds, Gina Carano, Kayla Adams, Randa...",Tim Miller,Ken Seng,"Paul Wernick, Fabian Nicieza, Rob Liefeld, Rhe...","Ryan Reynolds, Jonathon Komack Martin, Paul We...",Tom Holkenborg,8.0,1167674.0,"[-0.05629737302660942, 0.005468115210533142, -..."


In [6]:
# Top 1500
top_1500 = full_df.head(50000)
top_1500.to_pickle('../raw/top_50000.pkl')


In [6]:
top: pd.DataFrame = pd.read_pickle('../raw/top_50000.pkl')
top.head()


,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding
0,Inception,8.369,36269.0,Released,2010-07-15,8.255328e+08,148.0,160000000.0,tt1375666,en,...,"English, Français, 日本語, Kiswahili","Leonardo DiCaprio, Tim Kelleher, Jack Gilroy, ...",Christopher Nolan,Wally Pfister,Christopher Nolan,"Thomas Tull, Yoshikuni Taki, Christopher Nolan...",Hans Zimmer,8.8,2593927.0,"[-0.007635640446096659, -0.004168331157416105,..."
1,Interstellar,8.440,35193.0,Released,2014-11-05,7.017292e+08,169.0,165000000.0,tt0816692,en,...,English,"Russ Fega, Liam Dickinson, Jeff Hephner, Brook...",Christopher Nolan,Hoyte van Hoytema,"Jonathan Nolan, Christopher Nolan","Jake Myers, Jordan Goldberg, Lynda Obst, Chris...",Hans Zimmer,8.7,2159078.0,"[-0.0350022129714489, 0.023837948217988014, -0..."
2,The Dark Knight,8.516,32550.0,Released,2008-07-16,1.004558e+09,152.0,185000000.0,tt0468569,en,...,"English, 普通话","Matthew O'Neill, Nestor Carbonell, Morgan Free...",Christopher Nolan,Wally Pfister,"David S. Goyer, Jonathan Nolan, Bob Kane, Chri...","Kevin De La Noy, Michael Uslan, Thomas Tull, C...","Hans Zimmer, James Newton Howard",9.0,2921100.0,"[0.002534319879487157, -0.03606430068612099, -..."
3,Avatar,7.582,31265.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,en,...,"English, Español","Jason Whyte, Scott Lawrence, Brandon Jelkes, J...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,7.9,1402405.0,"[-0.028900334611535072, -0.0029624924063682556..."
4,Deadpool,7.621,30696.0,Released,2016-02-09,7.828373e+08,108.0,58000000.0,tt1431045,en,...,English,"Ryan Reynolds, Gina Carano, Kayla Adams, Randa...",Tim Miller,Ken Seng,"Paul Wernick, Fabian Nicieza, Rob Liefeld, Rhe...","Ryan Reynolds, Jonathon Komack Martin, Paul We...",Tom Holkenborg,8.0,1167674.0,"[-0.05629737302660942, 0.005468115210533142, -..."


In [7]:

# # Precompute norms for the movie embeddings to speed up cosine similarity
# self.df['embedding'] = self.df['embedding'].apply(np.array)
# self.df['embedding_norm'] = self.df['embedding'].apply(np.linalg.norm)
import numpy as np

top['embedding'] = top['embedding'].apply(np.array)
top['embedding_norm'] = top['embedding'].apply(np.linalg.norm)

top.head()


,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding,embedding_norm
0,Inception,8.369,36269.0,Released,2010-07-15,8.255328e+08,148.0,160000000.0,tt1375666,en,...,"Leonardo DiCaprio, Tim Kelleher, Jack Gilroy, ...",Christopher Nolan,Wally Pfister,Christopher Nolan,"Thomas Tull, Yoshikuni Taki, Christopher Nolan...",Hans Zimmer,8.8,2593927.0,"[-0.007635640446096659, -0.004168331157416105,...",1.0
1,Interstellar,8.440,35193.0,Released,2014-11-05,7.017292e+08,169.0,165000000.0,tt0816692,en,...,"Russ Fega, Liam Dickinson, Jeff Hephner, Brook...",Christopher Nolan,Hoyte van Hoytema,"Jonathan Nolan, Christopher Nolan","Jake Myers, Jordan Goldberg, Lynda Obst, Chris...",Hans Zimmer,8.7,2159078.0,"[-0.0350022129714489, 0.023837948217988014, -0...",1.0
2,The Dark Knight,8.516,32550.0,Released,2008-07-16,1.004558e+09,152.0,185000000.0,tt0468569,en,...,"Matthew O'Neill, Nestor Carbonell, Morgan Free...",Christopher Nolan,Wally Pfister,"David S. Goyer, Jonathan Nolan, Bob Kane, Chri...","Kevin De La Noy, Michael Uslan, Thomas Tull, C...","Hans Zimmer, James Newton Howard",9.0,2921100.0,"[0.002534319879487157, -0.03606430068612099, -...",1.0
3,Avatar,7.582,31265.0,Released,2009-12-15,2.923706e+09,162.0,237000000.0,tt0499549,en,...,"Jason Whyte, Scott Lawrence, Brandon Jelkes, J...",James Cameron,Mauro Fiore,James Cameron,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,7.9,1402405.0,"[-0.028900334611535072, -0.0029624924063682556...",1.0
4,Deadpool,7.621,30696.0,Released,2016-02-09,7.828373e+08,108.0,58000000.0,tt1431045,en,...,"Ryan Reynolds, Gina Carano, Kayla Adams, Randa...",Tim Miller,Ken Seng,"Paul Wernick, Fabian Nicieza, Rob Liefeld, Rhe...","Ryan Reynolds, Jonathon Komack Martin, Paul We...",Tom Holkenborg,8.0,1167674.0,"[-0.05629737302660942, 0.005468115210533142, -...",1.0


In [8]:
top.to_pickle('../raw/top_50000.pkl')